In [37]:
import sys
from mozaik.storage.datastore import PickledDataStore
from mozaik.controller import Global
from parameters import ParameterSet
from mozaik.controller import setup_logging
from mozaik.tools.export_to_hdf5 import *

def get_datastore(root):
    Global.root_directory = root
    setup_logging()
    
    datastore = PickledDataStore(
        load=True,
        parameters=ParameterSet({"root_directory": root, "store_stimuli": None}),
        replace=True,
    )
    return datastore

if __name__ == "__main__":
    simulation_path = sys.argv[1]
    datastore = get_datastore('/projects/urbanosz/multitrial/20241226-154716[param_nat_img.defaults]CombinationParamSearch{experiments.images_dir:[0],experiments.num_skipped_images:[200000],experiments.num_images:[50],experiments.num_trials:[100]}/V1_model_Dataset_ParameterSearch_____images_dir:0_num_images:50_num_skipped_images:200000_num_trials:100')
    datastore.print_content()
    #export_from_datastore_to_hdf5(data_store=datastore, st_name='StaticImage', data_type='mean_rates')

  DSV info:
  DSV info:
  DSV info:
  DSV info:
  DSV info:
  DSV info:
  DSV info:
     Number of recordings: 60012
     Number of recordings: 60012
     Number of recordings: 60012
     Number of recordings: 60012
     Number of recordings: 60012
     Number of recordings: 60012
     Number of recordings: 60012
       InternalStimulus : 12
       InternalStimulus : 12
       InternalStimulus : 12
       InternalStimulus : 12
       InternalStimulus : 12
       InternalStimulus : 12
       InternalStimulus : 12
       StaticImage : 60000
       StaticImage : 60000
       StaticImage : 60000
       StaticImage : 60000
       StaticImage : 60000
       StaticImage : 60000
       StaticImage : 60000
     Number of ADS: 0
     Number of ADS: 0
     Number of ADS: 0
     Number of ADS: 0
     Number of ADS: 0
     Number of ADS: 0
     Number of ADS: 0


In [38]:
dsv = param_filter_query(datastore,sheet_name='V1_Exc_L2/3')

dsv.print_content()

  DSV info:
  DSV info:
  DSV info:
  DSV info:
  DSV info:
  DSV info:
  DSV info:
     Number of recordings: 10002
     Number of recordings: 10002
     Number of recordings: 10002
     Number of recordings: 10002
     Number of recordings: 10002
     Number of recordings: 10002
     Number of recordings: 10002
       InternalStimulus : 2
       InternalStimulus : 2
       InternalStimulus : 2
       InternalStimulus : 2
       InternalStimulus : 2
       InternalStimulus : 2
       InternalStimulus : 2
       StaticImage : 10000
       StaticImage : 10000
       StaticImage : 10000
       StaticImage : 10000
       StaticImage : 10000
       StaticImage : 10000
       StaticImage : 10000
     Number of ADS: 0
     Number of ADS: 0
     Number of ADS: 0
     Number of ADS: 0
     Number of ADS: 0
     Number of ADS: 0
     Number of ADS: 0


In [39]:
import cProfile
def aaa(dsv):
    segs = dsv.get_segments()
    for i in range(0,10): 
        print(i)
        segs[i].get_spiketrains()
cProfile.run('aaa(dsv)', 'restats')    


0
Loading 33
1
Loading 34
2
Loading 35
3
Loading 36
4
Loading 37
5
Loading 38
6
Loading 39
7
Loading 40
8
Loading 41
9
Loading 42


In [9]:
def extract_sheet_data_and_save_to_h5py(stims, segs, varying_stim_params, data_type, stimuli_subgroup, sheet_name, cut_start, cut_end):
        # Get data to export
        logging.info(f"Extracting {data_type} data from {len(segs)} segments in sheet {sheet_name}")
        data = []
        for seg in segs:
            if data_type == 'mean_rates':
                if cut_start is not None:
                    cut_start = Quantity(cut_start, 'ms')
                if cut_end is not None:
                    cut_end = Quantity(cut_end, 'ms')
                data.append(seg.mean_rates(start=cut_start, end=cut_end))
            elif data_type == 'spiketrains':
                data.append([spiketrain.time_slice(cut_start, cut_end).magnitude for spiketrain in seg.get_spiketrains()])
            else:
                raise ValueError("Invalid data type")
            seg.release()

        data = np.array(data)

        # Reorder stimuli and data in tensors whose number of dimensions corresponds to the number of varying parameters
        stims_sorted, data_sorted = reorder_lists(stims, data, varying_stim_params.keys())

        # fix 2
        gc.collect()
        # stims_tensor = np.reshape(stims_sorted, [len(varying_stim_params[param]) for param in varying_stim_params.keys()])
        params_dims = [len(varying_stim_params[param]) for param in varying_stim_params.keys()]

        data_tensor = np.reshape(np.array(data_sorted).flatten(), [*params_dims, -1])

        # Add dataset to the stimuli subgroup
        sheet_name_cleaned = sheet_name.replace('/', '')
        if data_type =='spiketrains':
            dset = stimuli_subgroup.create_dataset(sheet_name_cleaned, shape=data_tensor.shape, dtype=h5py.special_dtype(vlen=np.dtype('float')))
            dset[:] = data_tensor
        else:
            stimuli_subgroup.create_dataset(sheet_name_cleaned, data=data_tensor)

In [10]:
def export_from_datastore_to_hdf5(data_store, st_name, data_type, start_time=None, stop_time=None, time_windows_size=None, path_to_save_hdf5=None):
    """
    Export data from a Mozaik datastore to a HDF5 file with a standardized structure.

    Parameters:
        data_store (DataStore): The Mozaik datastore containing simulation results
        st_name (str): The name of the stimulus to be exported
        data_type (str): The type of data to export. Options:
                        - 'mean_rates': Average firing rates
                        - 'spiketrains': Raw spike times
        start_time (float, optional): Start time (ms) for data extraction
        stop_time (float, optional): End time (ms) for data extraction

    Notes:
        Creates an HDF5 file with the following structure:
        - Root attributes: default_parameters, sim_info, sheets, data_type, st_name, recorders
        - Model groups: Contains parameter sets and stimulus data
        - Stimulus groups: Contains:
            - Constant/varying parameter information
            - Neural response data for each sheet
            - Stimulus data and indices
            - Data cut timing information

    """

    def serialize_parameters(params):
        serialized = {}
        for key, value in params.items():
            if isinstance(value, dict):
                serialized[key] = serialize_parameters(value)
            elif isinstance(value, PyNNDistribution):
                serialized[key] = str(value)
            else:
                serialized[key] = value
        return serialized
    
    def create_hdf5_structure(hf, sim_info, default_parameters, modified_parameters, recorders, experimental_protocols, st_name, varying_stim_params, constant_stim_params, data_type, start_time, stop_time):
        # Add default parameters and info as metadata to the group
        hf.attrs['default_parameters'] = str(serialize_parameters(default_parameters))
        hf.attrs['sim_info'] = str(sim_info)
        hf.attrs['data_type'] = data_type
        hf.attrs['st_name'] = st_name
        hf.attrs['recorders'] = str(recorders)
        hf.attrs['experimental_protocols'] = str(experimental_protocols)

        # Create a subgroup based on modified parameters
        if modified_parameters:
            model_subgroup_name = str(modified_parameters)
        else:
            model_subgroup_name = "default"
        model_subgroup = hf.create_group(model_subgroup_name)

        # Merge modified parameters into default parameters
        merged_parameters = default_parameters.copy()
        merged_parameters.update(modified_parameters)

        # Add merged parameters as metadata to the model_subgroup
        model_subgroup.attrs['parameters'] = str(serialize_parameters(merged_parameters))
        logging.info(f"Model subgroup '{model_subgroup_name}' created with merged parameters as metadata.")

        # Create a stimuli subgroup
        stimuli_subgroup = model_subgroup.create_group(st_name)
        logging.info(f"Datasets subgroup created under 'stimuli' in '{model_subgroup_name}'.")

        # Add varying parameters as metadata to the stimuli subgroup
        stimuli_subgroup.attrs['varying_parameters'] = list(varying_stim_params.keys())
        for param_name, param_values in varying_stim_params.items():
            stimuli_subgroup.attrs[f'{param_name}'] = param_values
        stimuli_subgroup.attrs['data_dimensions'] = [len(varying_stim_params[param]) for param in varying_stim_params.keys()]

        # Add constant parameters as metadata to the stimuli subgroup
        stimuli_subgroup.attrs['constant_parameters'] = list(constant_stim_params.keys())
        for param_name, param_values in constant_stim_params.items():
            stimuli_subgroup.attrs[f'{param_name}'] = str(param_values) if param_values is not None else "None"

        # Add data related metadata to the stimuli subgroup
        stimuli_subgroup.attrs['data_type'] = data_type
        stimuli_subgroup.attrs['data_start_time'] = str(start_time) if start_time is not None else "None"
        stimuli_subgroup.attrs['data_stop_time'] = str(stop_time) if stop_time is not None else "None"

        return stimuli_subgroup

    def get_segments_and_stimuli_and_constant_and_varying_parameters(data_store, sheet_name, st_name):
        # Get segments and stimuli
        dsv = param_filter_query(data_store, st_name=st_name, sheet_name=sheet_name)
        segs = dsv.get_segments()
        stims = [MozaikParametrized.idd(seg.annotations['stimulus']) for seg in segs]
        segs_pre =  dsv.get_segments(null=True)
        segs_post = [*segs_pre[1:], param_filter_query(data_store, st_name='InternalStimulus', sheet_name=sheet_name).get_segments()[-1]]
        segs = list(zip(segs_pre, segs, segs_post))

        # Get varying parameters
        constant_stim_params, varying_stim_params = classify_stimulus_parameters_into_constant_and_varying(stims)  # alternative: params = OrderedDict((param, sorted(list(parameter_value_list(stims, param)))) for param in varying_parameters(stims))

        # Assert all possible combinations of varying stimulus parameters are present in the stimuli in the list stims and that there are no duplicates
        assert len(stims) == np.prod([len(varying_stim_params[param]) for param in varying_stim_params.keys()]), "Number of stimuli does not match the product of the number of varying parameter values"
        assert len(set([str(stim) for stim in stims])) == len(stims), "There are duplicate stimuli"
        return segs, stims, constant_stim_params, varying_stim_params

    def extract_sheet_data_and_save_to_h5py(stims, segs, varying_stim_params, data_type, stimuli_subgroup, sheet_name, start_time, stop_time, time_windows_size):
        # Get data to export
        logging.info(f"Extracting {data_type} data from {len(segs)} segments in sheet {sheet_name}")

        # define start_time and stop_time if not provided
        if start_time is None:
            start_time = 0
        if stop_time is None:
            stop_time = stims[0].duration

        # define time_windows
        if time_windows_size == None: 
            time_windows = [(start_time, stop_time)]
            time_windows_size = stop_time - start_time
        else:
            assert float(stop_time-start_time) % (time_windows_size) == 0, "time_windows_size must be a multiple of the time_stop - time_start"
            time_windows = list(zip(np.arange(start_time, stop_time, time_windows_size), np.arange(start_time+time_windows_size, stop_time+time_windows_size, time_windows_size)))

        data = []
        for seg in segs:  
            concatenated_seg = concatenate_segments_with_offsets(*seg)
            if data_type == 'spike_counts':
                data.append(count_spikes_in_multiple_windows(concatenated_seg, time_windows))  
            elif data_type == 'mean_rates':
                data.append(count_spikes_in_multiple_windows(concatenated_seg, time_windows)/float(time_windows_size) * 1000)  
            elif data_type == 'spiketrains':
                data.append([spiketrain.time_slice(start_time, stop_time).magnitude for spiketrain in concatenated_seg.spiketrains])
            else:
                raise ValueError("Invalid data type")
        if data_type == 'spiketrains':
            data = np.array(data, dtype=object)
        else:
            data = np.stack(data)


        # Reorder stimuli and data in tensors whose number of dimensions corresponds to the number of varying parameters
        _, data_sorted = reorder_lists(stims, data, varying_stim_params.keys())

        params_dims = [len(varying_stim_params[param]) for param in varying_stim_params.keys()]
        # reshape data to match the dimensions of varying parameters
        data_tensor = np.reshape(np.array(data_sorted).flatten(), [*params_dims, *data_sorted[0].shape])
  
        # Add dataset to the stimuli subgroup
        sheet_name_cleaned = sheet_name.replace('/', '')
        if data_type =='spiketrains':
            dset = stimuli_subgroup.create_dataset(sheet_name_cleaned, shape=data_tensor.shape, dtype=h5py.special_dtype(vlen=np.dtype('float')))
            dset[:] = data_tensor
        else:
            stimuli_subgroup.create_dataset(sheet_name_cleaned, data=data_tensor)

    def add_stimuli_dataset(stimuli_subgroup, stims, varying_stim_params, ds):
        logging.info(f"Adding stimuli dataset to {stimuli_subgroup.name}")
        # Reorder stimuli and reshape to match the dimensions of varying parameters
        reordered_stims, _ = reorder_lists(stims, [str(s) for s in stims], varying_stim_params.keys()) 
        reordered_stims = np.array(reordered_stims).reshape([len(varying_stim_params[param]) for param in varying_stim_params.keys()])

        # Identify which dimension corresponds to trial
        trial_dim = None
        for i, param in enumerate(varying_stim_params.keys()):
            if param == 'trial':
                trial_dim = i
                stimuli_subgroup.attrs['trial_dim'] = trial_dim
                break

        # Drop the trial dimension by selecting the first element along it
        if trial_dim != None:
            reordered_stims = np.take(reordered_stims, 0, axis=trial_dim)
        # create index
        reordered_stims_flat = reordered_stims.flatten()
        reordered_stims_idx = np.arange(len(reordered_stims_flat)).reshape(reordered_stims.shape)
        # reinsert trial dimension
        if trial_dim != None:
            reordered_stims_idx = np.expand_dims(reordered_stims_idx, axis=trial_dim).repeat(len(varying_stim_params['trial']), axis=trial_dim)

        sensory_stim = np.array(ds.get_sensory_stimulus([str(s) for s in reordered_stims_flat])).squeeze()
        stimuli_subgroup.create_dataset('stimuli', data=sensory_stim)   
        stimuli_subgroup.create_dataset('stimuli_idx', data=reordered_stims_idx)

        # Save the stimulus_name dataset
        stimulus_names = np.array([str(stim) for stim in stims], dtype='S')
        stimuli_subgroup.create_dataset('stimulus_name', data=stimulus_names, dtype=h5py.string_dtype(encoding='utf-8'))

    ############################################################################################
    ## Create an HDF5 file (main function)
    ############################################################################################
    base_folder = data_store.parameters['root_directory']
    if path_to_save_hdf5 is None:
        path_to_save_hdf5 = os.path.join(base_folder, 'exported_data.h5')
    assert path_to_save_hdf5.endswith('.h5'), "path_to_save_hdf5 must end with .h5"
    os.makedirs(os.path.dirname(path_to_save_hdf5), exist_ok=True)
    with h5py.File(path_to_save_hdf5, 'w') as hf:
        # Get model info and parameters
        modified_parameters, default_parameters, info, recorders, experimental_protocols = get_model_info_and_parameters(base_folder, separate_modified_params=True)
        sheets =  data_store.sheets() 
        hf.attrs['sheets'] = [sheet.replace('/', '') for sheet in sheets]
        
        # Iterate over all sheets, extract data and save to h5py
        for i, sheet_name in enumerate(sheets):
            logging.info(f"Processing sheet {sheet_name}")
            # Get segments and stimuli and constant and varying parameters for the current sheet
            segs, stims, constant_stim_params, varying_stim_params = get_segments_and_stimuli_and_constant_and_varying_parameters(data_store=data_store, sheet_name=sheet_name, st_name=st_name)
            
            # Create HDF5 structure for the first sheet
            if i == 0:
                stimuli_subgroup = create_hdf5_structure(
                    hf, info, default_parameters, modified_parameters, recorders, experimental_protocols, st_name,
                    varying_stim_params, constant_stim_params, data_type, start_time, stop_time
                )

            if len(segs[0][1].get_spiketrains()) == 0: # maybe there is a better way to check if there are neurons recorded in the sheet
                logging.warning(f"No neurons recorded in sheet {sheet_name}")
                continue
            
            # # Extract data and save to h5py
            extract_sheet_data_and_save_to_h5py(stims, segs, varying_stim_params, data_type, stimuli_subgroup, sheet_name, start_time, stop_time, time_windows_size) 

        # Add stimuli dataset
        add_stimuli_dataset(stimuli_subgroup, stims, varying_stim_params, data_store)

    logging.info(f"HDF5 file created with default parameters, info, list of sheets as metadata, stimuli subgroup, and datasets subgroup.")

In [ ]:
export_from_datastore_to_hdf5(data_store=datastore, st_name='StaticImage', data_type='mean_rates')

In [30]:
import pstats
from pstats import SortKey
p = pstats.Stats('restats')
p.sort_stats(SortKey.CUMULATIVE).print_stats(20)

Tue Jan 14 14:11:39 2025    restats

         170375176 function calls (156848291 primitive calls) in 280.012 seconds

   Ordered by: cumulative time
   List reduced from 330 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       95    0.786    0.008  413.772    4.355 /usr/lib/python3.12/asyncio/base_events.py:1910(_run_once)
        1    0.000    0.000  306.092  306.092 {built-in method builtins.exec}
        1    0.000    0.000  306.090  306.090 /tmp/ipykernel_188518/4068454617.py:2(aaa)
       10    0.000    0.000  306.085   30.609 /home/antolikjan/virt_env/mozaikluca/lib/python3.12/site-packages/mozaik/storage/neo_neurotools_wrapper.py:47(get_spiketrains)
       10    0.003    0.000  266.664   26.666 /home/antolikjan/virt_env/mozaikluca/lib/python3.12/site-packages/mozaik/storage/neo_neurotools_wrapper.py:263(load_full)
       10    7.916    0.792  253.618   25.362 {built-in method _pickle.load}
   375000    1.998    0.000  245

In [12]:
import neo 
import quantities as qt 
import cProfile

def aaa():
    for i in range(3600000):
        neo.SpikeTrain(times=[]*qt.ms,t_stop=100)

cProfile.run('aaa()', 'restats_sp')    

In [40]:
import pstats
from pstats import SortKey
p = pstats.Stats('restats')
p.sort_stats(SortKey.CUMULATIVE).print_stats(20)

Tue Jan 14 14:38:44 2025    restats

         170375161 function calls (156848380 primitive calls) in 292.885 seconds

   Ordered by: cumulative time
   List reduced from 295 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      2/1    0.000    0.000  316.440  316.440 {built-in method builtins.exec}
        1    0.000    0.000  316.438  316.438 /tmp/ipykernel_188518/4068454617.py:2(aaa)
       10    0.000    0.000  316.433   31.643 /home/antolikjan/virt_env/mozaikluca/lib/python3.12/site-packages/mozaik/storage/neo_neurotools_wrapper.py:47(get_spiketrains)
       10    8.812    0.881  288.035   28.804 {built-in method _pickle.load}
   375000    2.147    0.000  251.341    0.001 /home/antolikjan/virt_env/mozaikluca/lib/python3.12/site-packages/neo/core/spiketrain.py:96(_new_spiketrain)
   375000    4.542    0.000  238.187    0.001 /home/antolikjan/virt_env/mozaikluca/lib/python3.12/site-packages/neo/core/spiketrain.py:260(__new__)
  